In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from numpy import arange
from sklearn.preprocessing import LabelBinarizer
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler


import warnings
warnings.simplefilter('ignore')

In [20]:
def calculate_metrics(df, crop_names, y_test, y_pred):
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    mape = metrics.mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test,y_pred)
    y_test_avg = pd.DataFrame(y_test).mean().iloc[0]
    y_pred_avg = pd.DataFrame(y_pred).mean().iloc[0]
    df.loc[crop_names] = [mae, mse, rmse, mape, r2, y_test_avg, y_pred_avg]


In [22]:
fao_data = pd.read_csv("datasets/latest0205.csv", index_col='Unnamed: 0')

fao_data

,Nutrient nitrogen N (total) - tonnes,Nutrient phosphate P2O5 (total) - tonnes,Nutrient potash K2O (total) - tonnes,Item,Yield(tonnes/ha),Fung & Bact – Benzimidazoles,"Fung & Bact – Diazines, morpholines",Fung & Bact – Dithiocarbamates,Fung & Bact – Inorganics,Fung & Bact – Other,...,Rodenticides – Other-3,Herbicides – Bipiridils-1,Herbicides – Bipiridils-2,Herbicides – Bipiridils-3,Herbicides – Sulfonyl ureas-1,Herbicides – Sulfonyl ureas-2,Herbicides – Sulfonyl ureas-3,Rodenticides – Anti-coagulants-1,Rodenticides – Anti-coagulants-2,Rodenticides – Anti-coagulants-3
1990-01-01,0.0,0.0,0.0,Apples,26.3272,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.00,5.00,5.00
1990-01-01,0.0,0.0,0.0,Apricots,2.5781,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.00,5.00,5.00
1990-01-01,0.0,0.0,0.0,Cherries,5.0828,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.00,5.00,5.00
1990-01-01,0.0,0.0,0.0,Currants,7.0383,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.00,5.00,5.00
1990-01-01,0.0,0.0,0.0,Gooseberries,6.8081,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.00,5.00,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-01,0.0,0.0,0.0,Sunflower seed,-0.8656,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,-81.0,0.0,0.0,31.0,0.41,0.41,0.41
2020-01-01,-106607.0,-55584.0,26854.0,Hop cones,0.2323,-15.0,-241.0,142.0,140.0,-773.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.41,0.41,0.41
2020-01-01,0.0,0.0,0.0,Rape or colza seed,1.4064,0.0,0.0,0.0,0.0,0.0,...,0.0,-84.0,0.0,0.0,4.0,0.0,0.0,0.10,0.41,0.41
2020-01-01,0.0,0.0,0.0,Soya beans,-1.0056,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-84.0,0.0,0.0,4.0,0.0,0.10,0.10,0.41


In [19]:
cat_names = fao_data['Category'].unique()
cat_names

array(['Fruits and Nuts', 'Vegetables and Legumes', 'Cereals',
       'Industrial_crops'], dtype=object)

In [7]:
quality_metrics = ['ANN_MAE', 'ANN_MSE', 'Random_forest_RMSE', 'ANN_MAPE','ANN_r2_score', 'y_test_avg', 'y_pred_avg']
error_df = pd.DataFrame(columns = quality_metrics, index = cat_names)
display(error_df)

,ANN_MAE,ANN_MSE,Random_forest_RMSE,ANN_MAPE,ANN_r2_score,y_test_avg,y_pred_avg
Fruits and Nuts,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vegetables and Legumes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cereals,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Industrial_crops,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
scaler = MinMaxScaler()
for cat in cat_names:
    crop_df = fao_data[fao_data['Category'] == cat]
    encoded_df = pd.get_dummies(crop_df[["Item"]], prefix='', prefix_sep='')

    y = crop_df[['Yield(tonnes/ha)']]
    x = crop_df.drop(['Yield(tonnes/ha)', "Item", "Category"], axis=1)
    x = pd.concat([x, encoded_df], axis=1)
    mean = x.mean()
    std = x.std()
    lower_bound = mean - 2 * std
    upper_bound = mean + 2 * std
    x = x.clip(lower_bound, upper_bound, axis=1)
    x = scaler.fit_transform(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)
    # Reshape the input data for LSTM
    n_features = x.shape[1] # Number of features in the input data
    X_train = x_train.reshape((x_train.shape[0], 1, n_features))
    X_test = x_test.reshape((x_test.shape[0], 1, n_features))
    # Define the model architecture
    model = tf.keras.Sequential([
    tf.keras.layers.LSTM(164, input_shape=(1, n_features), activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
    ])
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam())

    # Fit the model to the training data
    model.fit(X_train, y_train, epochs=1000, batch_size=32, verbose=1)

    #prediction of new value
    y_pred = model.predict(X_test)
    calculate_metrics(error_df, cat , pd.DataFrame(y_test), y_pred)

Epoch 1/1000
12/12 [==============================] - 2s 2ms/step - loss: 141.1039
Epoch 2/1000
12/12 [==============================] - 0s 2ms/step - loss: 95.1395
Epoch 3/1000
12/12 [==============================] - 0s 2ms/step - loss: 59.1925
Epoch 4/1000
12/12 [==============================] - 0s 2ms/step - loss: 57.6055
Epoch 5/1000
12/12 [==============================] - 0s 3ms/step - loss: 54.0454
Epoch 6/1000
12/12 [==============================] - 0s 3ms/step - loss: 52.8191
Epoch 7/1000
12/12 [==============================] - 0s 3ms/step - loss: 51.5681
Epoch 8/1000
12/12 [==============================] - 0s 2ms/step - loss: 51.0898
Epoch 9/1000
12/12 [==============================] - 0s 2ms/step - loss: 50.6899
Epoch 10/1000
12/12 [==============================] - 0s 2ms/step - loss: 48.6937
Epoch 11/1000
12/12 [==============================] - 0s 2ms/step - loss: 47.4675
Epoch 12/1000
12/12 [==============================] - 0s 4ms/step - loss: 46.0227
Epoch 13/100

KeyboardInterrupt: 

In [13]:
y_pred

array([[ 3.0949652 ],
       [-1.6309677 ],
       [ 0.7296394 ],
       [ 0.00962487],
       [-0.39650005],
       [-1.6241906 ],
       [ 0.32463542],
       [-0.29434463],
       [ 2.2305198 ],
       [-1.3764293 ],
       [ 0.78010565],
       [ 0.1641643 ],
       [-1.0365512 ],
       [-0.54548496],
       [ 0.1730462 ],
       [-1.0147219 ],
       [ 2.448805  ],
       [-1.9159803 ],
       [ 0.23783809],
       [ 0.05894116],
       [-2.0168571 ],
       [-0.97798234],
       [-0.803121  ],
       [-1.4865957 ],
       [ 1.897391  ],
       [-1.9970744 ],
       [-0.5761097 ],
       [-0.31250867],
       [-1.0586694 ],
       [-0.37014768],
       [-0.9375172 ],
       [-0.9476296 ],
       [ 2.0322337 ],
       [-1.5664849 ],
       [-0.8470572 ],
       [ 0.05337408],
       [-1.213195  ],
       [ 1.7366923 ],
       [-0.5651694 ],
       [-0.09223617],
       [-0.47371688],
       [ 0.8015561 ],
       [-0.787748  ],
       [-0.13930885],
       [ 0.53273296],
       [ 1

In [14]:
error_df

,ANN_MAE,ANN_MSE,Random_forest_RMSE,ANN_MAPE,ANN_r2_score,y_test_avg,y_pred_avg
Fruits and Nuts,2.189502,10.83248,3.291273,0.276856,0.786083,9.62286,9.215924
Vegetables and Legumes,11.86978,642.086022,25.339416,0.285387,0.819881,45.745672,37.134586
Cereals,0.579852,0.65248,0.807762,0.831017,0.888215,0.038645,0.046785
Industrial_crops,0.57666,0.490227,0.700162,1.486974,0.717725,0.030727,-0.171843
